# Route planner

> Module die alle functies bevat die nodig zijn om de snelste route te vinden langs een gegeven serie locaties.
> Deze maakt gebruik van de API van [openrouteservice](https://openrouteservice.org/dev/#/api-docs).
> Deze functies hoeven in principe niet door de gebruiker zelf te worden aangeroepen.

In [ ]:
#| default_exp route_get

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from datetime import datetime
import os

import pandas as pd
import numpy as np
from fastcore.utils import L

import openrouteservice
from geopy.geocoders import Nominatim

from project.data_get import get_data_from_azuresql
from project.utils import load_settings

## Openrouteservice API toelichting

We gebruiken de Openrouteservice API met de Python client (https://openrouteservice-py.readthedocs.io/en/latest/).
Voor het aantal berekeningen dat we moeten doen is toestemming nodig als je dat op de server van Openrouteservice zelf wil doen. Jelle heeft hier toestemming voor gerkegen. De API key is dus aan zijn account gekoppeld en niet aan WDODelta.

In [ ]:
#| hide
# only used for testing while developping
from fastcore.utils import Path
from project.data_get import load_pickle


In [ ]:
#| hide
# Only used for testing
settings = load_settings()

file_path = settings['files']['path_pickle_results']
file_name = "get_data_from_azuresql_20250124_130111.pickle"
peilbuizen_df = load_pickle(file_path=Path(file_path) / file_name)

df_grouped = peilbuizen_df.groupby('project')
df_grouped.groups.keys()

dict_keys(['---', 'Bagger Dedemsvaart', 'Baggeren Beilervaart', 'Beekmaatregelen Reest', 'Boetelerveld_nw', 'Brongeb VL-AA', 'Dijken', 'Droogte Onderzoek NL fase 2 - bodemvochtmetingen', 'Droogtemeetnet WDOD', 'Ecologisch Effect Beekherstel Middenloop Vledder Aa Fase 1', 'Holtingerveld _Ootmaanlanden en Koningschut', 'Inrichting Dwingelderveld', 'Koekoekspolder grondwatermeetnet', 'Mastenbroek-IJssel', 'Meetnet De Wijk II', 'Nieuwveense landen', 'Nijstad Hoogeveen', 'Olde Maten en Veerslootlanden', 'Oldematen Reevediep', 'Oude Diep', 'Oude Diep Mantinge Bos_Zand', 'Oude Diep_Roode Brand', 'Oude Willem', 'Overijsselskanaal Deventer-Raalte', 'Paddenpol Zwolle-Olst', 'Primair meetnet blok 1', 'Primair meetnet blok 2', 'Primair meetnet blok 3', 'Randzone Ossenzijl Steenwijk', 'Reevediep', 'Reparatie meetpunten', 'Steenwijk_Kallenkote', 'Varsenerveld', 'Vecht', 'Vechterweerd', 'Vledder en Leierhooilanden', 'Wabos-KRW', 'ZUIDWOLDE-ZUID', 'Zandwetering Olst-Zuid_waterberging', 'dijken Stadsdi

In [ ]:
#| hide
df_test = df_grouped.get_group('Boetelerveld_nw')

In [ ]:
#| export

client = openrouteservice.Client(key=os.environ["OPENROUTESERVICE_KEY"])

## Bepaald de coordinaten van de startlocatie.

We vinden hier hier de longitude en latitude coordinaten van de startlocatie die als adres kan worden opgegeven in de `settings.yaml` file.

We gebruiken hiervoor de Nominatim geocoder (https://python-geopy-homework.readthedocs.io/en/latest/geocoders/nominatim.html) en niet de Openrouteservice, omdat (voor zover Jelle weet) Openrouteservice deze optie niet biedt.

In [ ]:
#| export
def get_lonlat_start_location(address: str = "Dokter van Deenweg 186, 8025 BM, Zwolle"):
    """Get the longitude and latitude coordinates from a given adress"""
    geolocator = Nominatim(user_agent="wdodelta_route_optimizer", timeout=10)
    max_retries = 3
    retry_delay = 2

    for attempt in range(max_retries):
        try:
            location = geolocator.geocode(address)
            if location:
                return (location.longitude, location.latitude)
            else:
                print(f"Warning: Could not find coordinates for adress: {address}")
                return None
        except GeocoderTimedOut:
            if attempt < max_retries - 1:
                print(f"Warning: Geocoder timed out, attempt {attempt + 1}/{max_retries}. Retrying...")
                time.sleep(retry_delay)
            else:
                print(f"Error: Geocoder timed out after {max_retries} attempts")
                return None
        except GeocoderServiceError as e:
            print(f"Error: Geocoding service error - {str(e)}")
            return None
        except Exception as e:
            print(f"Error: Unexpected error during geocoding - {str(e)}")
            return None

## Haal de coordinaten van de peilbuislocaties op uit de peilbuizen dataframe.

In [ ]:
#| export
def df_to_longlat_tuple(df, longitude_column, latitude_column):
    """Get the longitude and latitude coordinates from all rows in a pandas dataframe
    and return a list of longitude, latitude tuples"""
    return [(row[longitude_column], row[latitude_column]) for _, row in df.iterrows()]

In [ ]:
#| hide
start_location = get_lonlat_start_location(settings['calculation']['startlocation'])
start_location

(6.1254037, 52.5069559)

In [ ]:
#| hide
len(df_test)

17

In [ ]:
#| hide
longlat_tpl_test = df_to_longlat_tuple(df_test, longitude_column="Longitude", latitude_column="latitude")
longlat_tpl_test

[(6.31447943904077, 52.3695816028906),
 (6.33588886886419, 52.3668413010561),
 (6.34241251218036, 52.3636339878603),
 (6.33914653000434, 52.3626446506742),
 (6.34038015617724, 52.353098309015),
 (6.30853883795403, 52.3684775306057),
 (6.34416955328974, 52.3606896598573),
 (6.32698979439397, 52.3633260588481),
 (6.34155389136252, 52.3544370677718),
 (6.31519312162391, 52.3615855946565),
 (6.34548591239569, 52.3550254656901),
 (6.34068514963815, 52.3631895256562),
 (6.31703124959358, 52.3708288993225),
 (6.3429287334774, 52.3623804695934),
 (6.34625596735498, 52.3587313286589),
 (6.31295601019247, 52.3627266312915),
 (6.31182784125961, 52.3557966399885)]

## Find the shortest route through all locations starting at the start location and ending at the start location. Using the openrouteservice API.

In [ ]:
#| export
def create_optimized_route(start_address: str, # De startlocatie en eindlocatie opgegeven als adres
                           df: pd.DataFrame, # Dataframe met peilbuizen en coordinaten in WGS84
                           route_profile: str, # Het route profiel (transport methode) waarvoor de route berekend moet worden
                           long_clmn: str="Longitude", # Dataframe kolomnaam met de longitude coordinaten in WGS84
                           lat_clmn: str="latitude" # Dataframe kolomnaam met de latitude coordinaten in WGS84
                           ):
    """Solve the traveling salesman problem (visit all given points exactly once in the 
    shortes possible route) for a given start adress and pandas dataframe with longitude
    and latitude columns."""
    start_coords = get_lonlat_start_location(start_address)
    peilbuizen_coords = df_to_longlat_tuple(df, longitude_column=long_clmn, latitude_column=lat_clmn)
    total_coords = L([start_coords] + peilbuizen_coords + [start_coords])
    return client.directions(total_coords,
                           profile=route_profile,
                           optimize_waypoints=True,
                           instructions=False,
                           geometry=True,
                           format='geojson',
                           preference='fastest',
                           radiuses=-1 # Don't restrict radius to search for routepoint near peilbuis,
                           
    )


In [ ]:
#| hide
route_profile = settings['calculation']['distance_calculation_method']
route_profile

'cycling-regular'

In [ ]:
#| hide
client.directions(longlat_tpl_test, 
                  profile=route_profile,
                  optimize_waypoints=True,
                  instructions=False,
                  geometry=True,
                  format='geojson',
                  preference='fastest',
                  radiuses=-1)

{'type': 'FeatureCollection',
 'bbox': [6.301719, 52.352912, 6.356489, 52.37279],
 'features': [{'bbox': [6.301719, 52.352912, 6.356489, 52.37279],
   'type': 'Feature',
   'properties': {'way_points': [0,
     10,
     42,
     48,
     49,
     50,
     64,
     71,
     77,
     83,
     85,
     123,
     178,
     207,
     218,
     230,
     242],
    'summary': {'distance': 19960.6, 'duration': 5088.1}},
   'geometry': {'coordinates': [[6.313255, 52.370191],
     [6.313427, 52.37032],
     [6.313226, 52.370424],
     [6.313197, 52.370511],
     [6.314538, 52.370706],
     [6.31524, 52.37081],
     [6.315513, 52.370873],
     [6.31581, 52.37098],
     [6.316328, 52.371234],
     [6.316668, 52.371399],
     [6.317259, 52.37098],
     [6.316668, 52.371399],
     [6.317595, 52.371882],
     [6.318057, 52.372121],
     [6.325381, 52.370716],
     [6.329314, 52.369958],
     [6.331422, 52.369548],
     [6.332195, 52.369398],
     [6.335619, 52.368733],
     [6.336381, 52.368598],
   

In [ ]:
#| hide
t = create_optimized_route(start_address=settings['calculation']['startlocation'],
                           df=df_test,
                           route_profile=route_profile)

t.keys()
len(t['features'][0]['properties']['way_points'])

19

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()